In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas_profiling import ProfileReport
%matplotlib inline
%load_ext nb_black


In [ ]:
initial_df = pd.read_csv("insurance.csv")
initial_df

In [ ]:
profile = ProfileReport(initial_df)
profile

In [ ]:
sns.pairplot(initial_df, hue='smoker')
plt.show()


In [ ]:
# one hot encoding
df = pd.get_dummies(initial_df, drop_first=True)
# dodanie kategorii otyły (w USA wg jakiegoś medycznego instytu jest po prostu powyżej 30
df["obese"] = df["bmi"].apply(lambda x: 1 if x >= 30 else 0)
# dodanie kategorii otyły i palący
df["obese_smoker"] = df[["obese", "smoker_yes"]].apply(
    lambda x: 1 if (x["obese"] == 1) & (x["smoker_yes"] == 1) else 0, axis=1
)

In [ ]:
# sprawdzenie
df

In [ ]:
# nowa mapa korelacji
corr = np.corrcoef(np.random.randn(11, 200))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(12, 12))
sns.heatmap(df.corr(), cmap="YlGnBu", mask=mask, square=True)

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x=initial_df["age"])

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x=initial_df["charges"], whis=3.0)

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(y=df["charges"], x=df["obese_smoker"], whis=1.5)

In [ ]:
ax = sns.boxplot(x="day", y="total_bill", hue="smoker",
                 data=tips, palette="Set3")

In [ ]:
df[df["charges"] > 50000]

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x=initial_df["bmi"], whis=1.5)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop("charges", axis=1)
y = df["charges"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
lin_reg.score(X_test, y_test)


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
pipeline = make_pipeline(ss, lin_reg)
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipeline, X_train, y_train, cv=5).mean()